In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

2023-04-10 19:36:19.242985: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
import string
from string import digits

In [11]:
# Load the data
data=pd.read_json('/Users/vanord/Desktop/alphabet/week2/productRatings/reviews.json',lines=True)
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(data['reviewText'], data['overall'], test_size=0.2, random_state=42)

In [32]:
# Preprocess the text data
stop_words = set(stopwords.words('english'))
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens=[i.translate(str.maketrans('', '', digits)) for i in tokens]
    return ' '.join(tokens)

In [33]:
X_train_del = X_train_o.apply(preprocess)
X_test_del = X_test_o.apply(preprocess)

In [34]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_del)

In [35]:
X_train = tokenizer.texts_to_sequences(X_train_del)
X_test = tokenizer.texts_to_sequences(X_test_del)

vocab_size = len(tokenizer.word_index) + 1

In [37]:
# Pad the sequences to a fixed length
max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

In [38]:
X_train.shape

(237069, 100)

In [39]:
# Train the Word2Vec model
sentences = [sentence.split() for sentence in X_train_del]
w2v_model = Word2Vec(sentences, window=5, min_count=5, workers=4)

In [40]:
# Create a weight matrix for the embedding layer
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

In [45]:
# Define the CNN model
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train_o, epochs=10, batch_size=32, validation_data=(X_test, y_test_o))

Epoch 1/10
7409/7409 [==============================] - 109s 15ms/step - loss: 0.8589 - accuracy: 0.6649 - val_loss: 0.8246 - val_accuracy: 0.6757
Epoch 2/10
7409/7409 [==============================] - 114s 15ms/step - loss: 0.8040 - accuracy: 0.6812 - val_loss: 0.8110 - val_accuracy: 0.6824
Epoch 3/10
7409/7409 [==============================] - 111s 15ms/step - loss: 0.7757 - accuracy: 0.6901 - val_loss: 0.8032 - val_accuracy: 0.6837
Epoch 4/10
7409/7409 [==============================] - 108s 15ms/step - loss: 0.7522 - accuracy: 0.6981 - val_loss: 0.8132 - val_accuracy: 0.6799
Epoch 5/10
7409/7409 [==============================] - 107s 14ms/step - loss: 0.7304 - accuracy: 0.7057 - val_loss: 0.8131 - val_accuracy: 0.6792
Epoch 6/10
7409/7409 [==============================] - 112s 15ms/step - loss: 0.7099 - accuracy: 0.7123 - val_loss: 0.8270 - val_accuracy: 0.6759
Epoch 7/10
7409/7409 [==============================] - 119s 16ms/step - loss: 0.6909 - accuracy: 0.7211 - val_loss: 0